In [0]:
import keras
from keras import backend as K
import tensorflow as tf

In [0]:
n_class = 21

In [0]:
input_img = keras.layers.Input(shape=(473,473,3))

conv1 = keras.layers.Conv2D(filters=1024, kernel_size=(5,5), padding='same', data_format='channels_last', strides=3)(input_img)
bn1 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv1)
act1 = keras.layers.Activation(activation='relu')(bn1)
pool1 = keras.layers.MaxPool2D((2,2))(act1)

conv2 = keras.layers.Conv2D(filters=2048, kernel_size=(4,4), padding='same', data_format='channels_last', strides=2)(pool1)
bn2 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv2)
act2 = keras.layers.Activation(activation='relu')(bn2)
pool2 = keras.layers.MaxPool2D((2,2))(act2)

conv3 = keras.layers.Conv2D(filters=3072, kernel_size=(3,3), padding='same', data_format='channels_last')(pool2)
bn3 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv3)
act3 = keras.layers.Activation(activation='relu')(bn3)
pool3 = keras.layers.AveragePooling2D((2,2))(act3)

conv4 = keras.layers.Conv2D(filters=4096, kernel_size=(2,2), padding='same', data_format='channels_last')(pool3)
bn4 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv4)
act4 = keras.layers.Activation(activation='relu')(bn4)
pool4 = keras.layers.AveragePooling2D((2,2))(act4)

channel_fusion = keras.layers.Conv2D(filters=1000, kernel_size=(1,1), padding='same', data_format='channels_last')(pool4)

up1 = keras.layers.UpSampling2D((2,2))(channel_fusion)
conv5 = keras.layers.Conv2D(filters=4096, kernel_size=(4,4), padding='same', data_format='channels_last')(up1)
bn5 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv5)
act5 = keras.layers.Activation(activation='relu')(bn5)

up2 = keras.layers.UpSampling2D((3,3))(act5)
conv6 = keras.layers.Conv2D(filters=3072, kernel_size=(2,2), padding='same', data_format='channels_last')(up2)
bn6 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv6)
act6 = keras.layers.Activation(activation='relu')(bn6)

up3 = keras.layers.UpSampling2D((4,4))(act6)
conv7 = keras.layers.Conv2D(filters=2048, kernel_size=(3,3), padding='same', data_format='channels_last')(up3)
bn7 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv7)
act7 = keras.layers.Activation(activation='relu')(bn7)

up4 = keras.layers.UpSampling2D((2,2))(act7)
conv8 = keras.layers.Conv2D(filters=1024, kernel_size=(4,4), padding='valid', data_format='channels_last')(up4)
bn8 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv8)
act8 = keras.layers.Activation(activation='relu')(bn8)

up5 = keras.layers.UpSampling2D((2,2))(act8)
conv9 = keras.layers.Conv2D(filters=n_class, kernel_size=(2,2), padding='valid', data_format='channels_last')(up5)
bn9 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv9)
output_img = keras.layers.Activation(activation='softmax')(bn9)

In [25]:
FCNN = keras.models.Model(input_img, output_img)
print(FCNN.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 473, 473, 3)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 158, 158, 1024)    77824     
_________________________________________________________________
batch_normalization_19 (Batc (None, 158, 158, 1024)    4096      
_________________________________________________________________
activation_19 (Activation)   (None, 158, 158, 1024)    0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 79, 79, 1024)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 40, 40, 2048)      33556480  
_________________________________________________________________
batch_normalization_20 (Batc (None, 40, 40, 2048)      8192

In [0]:
FCNN.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics=['categorical_accuracy'])

In [0]:
history = FCNN.fit(
            x_train, y_train,
            epochs = 15,
            batch_size = 64,
            validation_data = [x_test,y_test])

In [0]:
y_hat = FCNN.predict(x_test)

In [0]:
# Saved in the d2l package for later use
VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]
# Saved in the d2l package for later use
VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person',
               'potted plant', 'sheep', 'sofa', 'train', 'tv/monitor']

In [22]:
print(len(VOC_COLORMAP))

21


In [26]:
print(len(VOC_CLASSES))

21


In [0]:
# 以下位predict数据的可视化

In [0]:
class_dic = {}
for i,j in enumerate(VOC_CLASSES):
    class_dic[i] = j

In [0]:
y_hat_argmax = []
temp_row = []
temp_img = []
# y_hat => (-1, 473, 473, 21)
for img in y_hat:
    # img => 473,473,21
    for rows in img:
        for pixel in rows:
            # pixel => (1,1,21)
            temp_row.append(np.argmax(pixel[:,2]))
        temp_img.append(temp_row)
        temp_row = []
    y_hat.append(temp_img)
    temp_img = []
# by now, all pixels in y_hat_argmax will have only one most likely class

In [0]:
from PIL import Image

In [0]:
for imgs in y_hat_argmax:
    c = Image.new("RGB", (473, 473))
    for i in range(473):
        for j in range(473):
            c.putpixel([i,j],VOC_COLORMAP[imgs[i,j]])
    c.show()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# https://d2l.ai/chapter_computer-vision/fcn.html
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html#display-image
# 上列两个网站可以给如何处理图像数据提供参考

In [0]:
import os
import cv2

In [0]:
b,g,r = cv2.split(img)

In [0]:
# this function is for read image,the input is directory name
def read_directory(directory_name):
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(directory_name):
        #print(filename) #just for test
        #img is used to store the image data 
        img = cv2.imread(directory_name + "/" + filename)
        array_of_img.append(img)
        #print(img)
        # print(array_of_img)
    
    return array_of_img

In [0]:
array_of_img = [] 

In [0]:
array_of_img = read_directory("/content/drive/My Drive/dataset1/images_prepped_train")